In [14]:
import pandas as pd

In [15]:
# Function to calculate Roll's spread estimator for groups
def rolls_estimator(group):
    rates = group['Rate 1'].diff().dropna()
    cov_rate = rates.cov(rates.shift())
    return 2 * (abs(cov_rate) ** 0.5) if cov_rate < 0 else 0

In [16]:
df = pd.read_excel('data/liquidity/CAD_20130223-20130310.xlsx')
df.head()
df  = df.sort_values(by=['Trade Time'])

In [17]:
df['Trade Date'] = pd.to_datetime(df['Trade Time']).dt.date
df['Tenor_Years'] = (pd.to_datetime(df['Maturity']) - pd.to_datetime(df['Effective'])).dt.days / 365.25
df['Tenor_Years_Rounded'] = df['Tenor_Years'].round()

In [18]:
# Apply filters
filtered_df = df[
    (df['Type'] == 'IRS Fix-Float') &
    (df['CD'] == 'TR') &
    ((df['Leg 1'] == 'FIXED') | (df['Leg 1'] == 'CAD-BA-CDOR')) &
    (df['Leg 2'].isin(['FIXED', 'CAD-BA-CDOR', ''])) &
    (df['PF 1'] != '1T') &
    (df['PF 2'] != '1T') &
    (df['Curr'] == 'CAD') &
    (df['Othr Pmnt'].isnull()) &
    (df['Rate 2'].isnull())
]

In [19]:
grouped = filtered_df.groupby(['Tenor_Years_Rounded', 'Trade Date'])

In [20]:
grouped.apply(rolls_estimator).reset_index(name='Roll_Measure')

/Users/arnobalam/opt/anaconda3/lib/python3.9/site-packages/pandas/core/nanops.py:1609: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/Users/arnobalam/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,Tenor_Years_Rounded,Trade Date,Roll_Measure
0,1.0,2013-02-25,0.066755
1,1.0,2013-02-26,0.000000
2,1.0,2013-02-27,0.155931
3,1.0,2013-02-28,0.000000
4,1.0,2013-03-01,1.140430
...,...,...,...
90,30.0,2013-03-01,0.000000
91,30.0,2013-03-04,0.000000
92,30.0,2013-03-06,0.000000
93,30.0,2013-03-07,0.000000


In [21]:
grouped.size()

Tenor_Years_Rounded  Trade Date
1.0                  2013-02-25     4
                     2013-02-26     1
                     2013-02-27    62
                     2013-02-28     9
                     2013-03-01    10
                                   ..
30.0                 2013-03-01     1
                     2013-03-04     2
                     2013-03-06     4
                     2013-03-07     2
                     2013-03-08     1
Length: 95, dtype: int64